In [3]:
import numpy as np
import pandas as pd 
import os 
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
parent_dir=os.path.dirname(os.getcwd())

def read_line_emission_csv(path_line_emission_csv):
    """
    Read the csv file in which are stored the line emission's rest frequency.
    
    Parameter: 
    path_line_emission_csv (str): Path to file.csv within there are the line emission's rest frequency.
    
    Return:
    pd.DataFrame : Dataframe with line names and rest frequencies.
    """
    db_line = pd.read_csv(path_line_emission_csv, sep = ";")
    return db_line

def sed_reading(type_, path):
    cosmo = FlatLambdaCDM(H0=70 * U.km / U.s / U.Mpc, Tcmb0=2.725 * U.K, Om0=0.3)
    if type_ == "extended":
        file_path = path + "/sed_low_z_warm_star_forming_galaxy.dat"
        redshift = 10**(-4)
    elif type_ == "point":
        file_path = path + "/sed_low_z_type2_AGN.dat"
        redshift = 0.05
    else:
        return "Not valid type"
    
    try: 
        SED= pd.read_csv(file_path, sep="\s+")
        rename_columns = {
            'um' : 'GHz',
            'erg/s/Hz': 'Jy',
        }
        SED.rename(columns=rename_columns, inplace=True)
        SED['GHz']=SED['GHz'].apply(lambda x: (x* U.um).to(U.GHz, equivalencies=U.spectral()).value)
        if type_ == 'point': 
            SED['Jy']=SED['Jy']/((10.**(-26.))*(10.**7.)*4.*np.pi*(cosmo.luminosity_distance(redshift).value*(3.086e+22))**2.)*(3.846e+33*1e+10)
        else:
            SED['Jy']=SED['Jy']/((10.**(-26.))*(10.**7.)*4.*np.pi*(cosmo.luminosity_distance(redshift).value*(3.086e+22))**2.)*(3.846e+33*1e+9)
        return SED
    except FileNotFoundError:
         return "File not Found"

def continuum_finder(SED,line_frequency):
    continuum_frequencies=SED['GHz'].values
    distances = np.abs(continuum_frequencies - np.ones(len(continuum_frequencies)*line_frequency))
    idx_=np.where(distances=np.min(distances))
    return np.log(SED['Jy'].values[idx_])

def cont_to_line(row):
    return np.exp(row['log_brightnes']+row['c'])

def process_spectral_data(type_, redshift, central_frequency, delta_freq, source_frequency, line_names=None ,n_lines=None):
    """
    Process spectral data based on the type of source, wavelength conversion,
    line ratios, and given frequency bands.
    
    Prameters:
    
    redshift: Redshift value to adjust the spectral lines and continuum.
    central_frequency: Central frequency of the observation band (GHz).
    delta_freq: Bandwidth around the central frequency (GHz).
    source_frequency: Frequency of the source obtained from metadata (GHz).
    lines: Optional list of line names provided by the user.
    n_lines: Number of additional lines to consider if lines is None.

    Output:


    """
    # Define the frequency range based on central frequency and bandwidth
    freq_min = central_frequency - delta_freq / 2
    freq_max = central_frequency + delta_freq / 2
    # Example data: Placeholder for continuum and lines from SED processing
    SED=sed_reading(type_,os.path.join(parent_dir,'brightnes'))

    # Placeholder for line data: line_name, observed_frequency (GHz), line_ratio, line_error
    db_line = read_line_emission_csv(os.path.join(parent_dir,'brightnes','Calibrations_FIR(GHz).csv'))
    # Shift the continuum and line frequencies by (1 + redshift)
    SED['GHz'] *= (1 + redshift)
    db_line['freq(GHz)'] = db_line['freq(GHz)'] * (1 + redshift)

    # Filter the continuum and lines to only include those within the frequency range
    continuum_mask = (SED['GHz'] >= freq_min) & (SED['GHz'] <= freq_max)
    filtered_continuum_frequencies = SED[continuum_mask]

    line_mask = (db_line['freq(GHz)'].astype(float) >= freq_min) & (db_line['freq(GHz)'].astype(float) <= freq_max)
    filtered_lines = db_line[line_mask]
    if len(filtered_lines) == 0:
        print('Warning: No lines fall in the selected band.')
    
    if line_names != None:
        user_lines = filtered_lines[np.isin(filtered_lines[:, 0], line_names)]
        if len(user_lines) == 0:
            print('Warning: Selected lines do not fall in the provided band, automaticaly computing most probable lines.')
        else:
            filtered_lines = user_lines
    
    filtered_lines['distance'] = np.abs(filtered_lines[:, 1].astype(float) - source_frequency)
    filtered_lines.sort_values(by='distance', inplace=True)
    if n_lines != None:
        if n_lines > len(filtered_lines):
            print(f'Warning: Can not insert {n_lines}, injecting {len(filtered_lines)}.')
        else:
            filtered_lines = filtered_lines.head(n_lines)
    
    filtered_lines['log_brightnes'] = filtered_lines['GHz'].apply(lambda x: continuum_finder(SED,x))
    brightnesses = filtered_lines.apply(cont_to_line, axis=1) 


    #Output the processed arrays and line information
    return filtered_continuum_frequencies, brightnesses, filtered_lines['Line']
